In [4]:
import gym
import numpy as np
import scipy.integrate as si
import matplotlib.pyplot as plt
from stable_baselines3 import PPO, DDPG
from stable_baselines3.common.env_checker import check_env

In [7]:
class Swing(gym.Env):
    g = 10

    def __init__(
        self,
        L0: float = 1.0,
        phi_0: float = 10.0,
        omega_0: float = -1.0,
        target_phi: float = np.pi / 2,
        dt: float = 0.01,
    ):
        super(Swing, self).__init__()
        self.L0 = L0
        self.Lmax = 2 * L0
        self.phi_0 = np.deg2rad(phi_0)
        self.omega_0 = omega_0
        self.phi = []
        self.phi.append(self.phi_0)
        self.omega = []
        self.omega.append(self.omega_0)
        self.L = []
        self.L.append(self.L0)  # so I can compute first derivative
        self.target_phi = target_phi
        self.pumps = 0
        self.action_space = gym.spaces.Box(low=0, high=1, shape=(1,))
        self.observation_space = gym.spaces.Box(low=-10, high=10, shape=(2,))

    def evolve(self):
        L_dot = (self.L[-1] - self.L[-2]) / self.dt
        omega_f = self.omega[-1] - self.dt * (
            2 * L_dot / self.L[-1] * self.omega[-1]
            + g / self.L[-1] * np.sin(self.phi[-1])
        )
        self.omega.append(omega_f)
        phi_new = self.phi[-1] + self.dt * self.omega_f
        self.phi.append(phi_new)
        self.pumps += 1

    def step(self, action):
        """
        Action is a choice of L
        """
        l_new = (action + 0.01) * self.Lmax
        self.L.append(l_new)
        self.evolve()
        state = np.array([self.phi[-1], self.omega[-1]])

        if np.isclose(self.phi[-1], self.target_phi, rtol=0.1):
            reward = 10
            done = True
            state = self.reset()
        else:
            reward = -1
            done = False
            if self.pumps > 15:
                done = True
                state = self.reset()
        info = _
        return state, reward, done, info

    def reset(self):
        self.clear()
        self.phi.append(self.phi_0)
        self.omega.append(self.omega_0)
        self.L.append(self.L0)
        state = np.array([self.phi[-1], self.omega[-1]])
        return state

    def render():
        pass

    def clear(self):
        self.L.clear()
        self.omega.clear()
        self.phi.clear()

In [8]:
check_env(Swing)

AssertionError: Your environment must inherit from the gym.Env class cf https://github.com/openai/gym/blob/master/gym/core.py

In [9]:
swing = Swing()

In [11]:
type(swing)

__main__.Swing